# Physical units

All variables in Scipp have a physical unit.
Variables are used for coordinates, data, and attributes, therefore, all of these have a unit.

## Basic Operations

Units are encoded by the [scipp.Unit](../generated/classes/scipp.Unit.rst) class.
Instances of this class can be constructed from strings:

In [ ]:
import scipp as sc

length = sc.Unit('m')
length

[scipp.Unit](../generated/classes/scipp.Unit.rst) defines mathematical operators for combining units:

In [ ]:
area = length * length
area

In [ ]:
volume = length * length * length
volume

In [ ]:
also_volume = length**3
also_volume

In [ ]:
sc.Unit('dimensionless') / length

In [ ]:
speed = length / sc.Unit('s')
speed

Invalid operations raise exceptions:

In [ ]:
speed + length

It is also possible to construct composite units directly from strings:

In [ ]:
sc.Unit('km')

In [ ]:
sc.Unit('m/s')

In [ ]:
sc.Unit('counts')

In [ ]:
sc.Unit('kg*m^2/s^2')

For convenience, the [scipp.units](../generated/modules/scipp.units.rst) module provides some frequently used units.
See [scipp.units](../generated/modules/scipp.units.rst) for a list of those units.

In [ ]:
sc.units.kg

In [ ]:
sc.units.m / sc.units.s

In [ ]:
sc.units.dimensionless

Use `repr` to see a unit in terms of SI (plus extensions) base units:

In [ ]:
repr(sc.Unit('V/L'))

This is especially helpful when it is unclear what a particular unit represents.

## Constructing Variables with Units

[Variables](../generated/classes/scipp.Variable.rst#scipp.Variable) with units can be constructed using the `units` argument in the constructor or in [creation functions](./creation-functions.rst).
When not specified explicitly, the unit of a variable usually (see below) defaults to `dimensionless` (a.k.a. `one`).
That is, the variable is considered dimensionless in terms of units (not to be confused with array dimensions).

In [ ]:
# same as sc.array(dims=['x'], values=[1, 2])
# and     sc.array(dims=['x'], values=[1, 2], unit='dimensionless')
sc.array(dims=['x'], values=[1, 2], unit='one')

In [ ]:
sc.array(dims=['x'], values=[1, 2], unit='m')

In [ ]:
sc.array(dims=['x'], values=[1, 2], unit=sc.units.m)

In [ ]:
sc.arange('x', 0, 3, unit=sc.units.s)

Scalars can also be constructed using multiplication or division of a number and a unit (in addition to [scipp.scalar](../generated/functions/scipp.scalar.rst#scipp.scalar)):

In [ ]:
1.2 * sc.Unit('kg/m^3')

In [ ]:
3.4 / sc.units.K

## Variables Without Units

It is not always meaningful to assign a unit to a variable.
For example, what is the unit of a string or a truth value?
For this reason, Scipp allows variables to have no unit by setting `unit=None`:

In [ ]:
sc.array(dims=['x'], values=[2, 4, 6], unit=None)

For non-numeric dtypes, the unit defaults to `None`:

In [ ]:
sc.array(dims=['x'], values=[False, True, False])

In [ ]:
sc.scalar('a string')

Indices are also non-physical quantities, so they should typically be defined without a unit, too.
To help with this, Scipp provides `sc.index`:

In [ ]:
sc.index(123)

Variables without units can interoperate like variables with units.
But the two groups cannot be combined:

In [ ]:
sc.scalar(1, unit='one') * sc.scalar(2, unit=None)

## Supported Units

Scipp supports a great number of units through LLNL's [Units](https://units.readthedocs.io/en/latest/index.html) library.
See in particular [Defined Units](https://units.readthedocs.io/en/latest/user-guide/defined_units.html).

<div class="alert alert-info">
     <b>INFO</b>

The LLNL/Units library is considered an implementation detail of Scipp.
Using SI units is safe but other unit systems should be used with discretion.
This applies especially to non-standard units like LLNL/Unit's custom (counting) units.
</div>

### Base Units
All SI base units are supported with the following names:

| Name  | Unit     |
|-------|----------|
| 'm'   | meter    |
| 's'   | second   |
| 'kg'  | kilogram |
| 'K'   | kelvin   |
| 'A'   | ampere   |
| 'mol' | mole     |
| 'cd'  | candela  |

In addition, the following base units are supported for cases not covered by SI.

| name    | Unit                   |
|---------|------------------------|
| 'rad'   | radian                 |
| 'count' | single object counting |

### Derived units
Many derived units can also be specified as arguments to `sc.Unit`.
Some examples are

| Name             | Unit          |
|------------------|---------------|
| 'Hz'             | hertz         |
| 'J'              | joule         |
| 'V'              | volt          |
| 'W'              | watt          |
| 'angstrom' / 'Å' | ångström     |
| 'eV'             | electron volt |
| 'L'              | liter         |
| 'min'            | minute        |
| 'D' / 'day'      | day           |

Units can be modified with SI prefixes, for instance

In [ ]:
print(
    sc.Unit('mm'),
    sc.Unit('microsecond'),
    sc.Unit('micro s'),
    sc.Unit('us'),
    sc.Unit('MJ'),
)

You can also specify exponents for units or exponentiate the `Unit` object:

In [ ]:
print(sc.Unit('m^2'), sc.Unit('m**2'), sc.Unit('m') ** 2)

## Conversion Between Units of Different Scales

Data can be converted between compatible units using [sc.to_unit](../generated/functions/scipp.to_unit.rst#scipp.to_unit).
Only conversions between units of the same physical dimensions are possible.

In [ ]:
sc.to_unit(1.0 * sc.units.m, 'mm')

In [ ]:
sc.to_unit(1.0 * sc.Unit('parsec'), 'm')

In [ ]:
sc.to_unit(3.14 * sc.Unit('m/s'), 'km/h')

In [ ]:
sc.to_unit(1.0 * sc.Unit('s'), 'm')

## Unit Aliases

It is possible to define custom aliases for units.
This can be used to

- guide string formatting to prefer a certain unit, e.g. angstrom over nm
- define domain specific units that can be expressed in terms of other units to
  - guide string formatting
  - construct units from strings with custom names
  
### Prioritizing Units in String Formatting

When dealing with crystals or molecules, it is often convenient to use angstrom as a unit.
But by default, string formatting tends to prefer different bases in composite units.
For example

In [ ]:
sc.Unit('us/angstrom**2')

This result is not very useful.
We can make the formatter prefer angstrom by defining it as an alias of itself (or alternatively of `'10^-10 m'`):

In [ ]:
sc.units.aliases['angstrom'] = 'angstrom'
sc.Unit('us/angstrom**2')

Note that 'angstrom' is predefined and simple units involving it, such as `sc.Unit('angstrom/s')` are formatted properly without the alias.
But the alias improves more complicated cases like the one above.

Aliases are global and stay in effect until they are removed.
This can be done using `sc.units.aliases.clear()` to remove all aliases or, to remove only one, using

In [ ]:
del sc.units.aliases['angstrom']
sc.Unit('us/angstrom**2')

Alternatively, a context manager can be used to remove aliases automatically:

In [ ]:
with sc.units.aliases.scoped(angstrom='angstrom'):
    print(sc.Unit('us/angstrom**2'))
print(sc.Unit('us/angstrom**2'))

But note that the context manager uses the global alias table and affects code outside of the context.
Details are explained in [scipp.units.aliases.scoped](../generated/modules/scipp.units.UnitAliases.rst#scipp.units.UnitAliases.scoped).

See [scipp.units.UnitAliases](../generated/modules/scipp.units.UnitAliases.rst) for the full API of `sc.units.aliases`.

### Defining New Units

It is possible to define completely new units as aliases as long as they can be expressed in terms of other units.
For example, define an speed unit:

In [ ]:
sc.units.aliases['speed'] = 'm/s'

In practice, a unit alias might refer to a characteristic speed, length, or time in a concrete physical system.
Or it might refer to a customary unit in a scientific field.

After defining the alias, the string formatter prioritizes 'speed' over 'm/s' when appropriate:

In [ ]:
print(sc.Unit('m/s'))
print(sc.Unit('km/s'))
print(sc.Unit('kg*m/s'))

And we can also construct units using 'speed' as an argument:

In [ ]:
print(sc.Unit('speed'))
print(sc.Unit('kg*mspeed**2'))

This also works in variables:

In [ ]:
sc.scalar(4, unit='speed')

Using `repr`, we can see that 'speed' is not a fundamentally new unit but simply expressed in terms of 'm' and 's':

In [ ]:
repr(sc.Unit('speed'))

### Defining Scaled Units

In the previous example, 'speed' was defined as a combination of 'm' and 's'.
It is also possible to define aliases of scaled units.
Examples of scaled units are millisecond, hour, or the previously used angstrom.
These units work by encoding a scale factor ('multiplier') in the unit.
This multiplier can be any floating point number, so for example, we can define a dog year as 52 days or 4492800 seconds:

In [ ]:
sc.units.aliases['dogyear'] = sc.scalar(4492800, unit='s')

Now we can use 'dogyear' as a unit:

In [ ]:
sc.Unit('dogyear')

We can see that the above is indeed a scaled unit by using `repr`:

In [ ]:
repr(sc.Unit('dogyear'))

Alternatively, we can specify the multiplier in the unit directly:

In [ ]:
sc.Unit('4492800s') == sc.Unit('dogyear')

We can use this to define the same alias as before but without going through a Scipp variable:

In [ ]:
sc.units.aliases.clear()
sc.units.aliases['dogyear'] = '4492800s'

In [ ]:
repr(sc.Unit('dogyear'))

Note that the unit multiplier is not the same as a value in a variable:

In [ ]:
var = sc.scalar(2, unit='dogyear')
var

Removing the alias reveals the multiplier (hover the mouse over the unit if it is abbreviated):

In [ ]:
del sc.units.aliases['dogyear']
var

The multiplier can be moved into the value of the variable by converting to seconds:

In [ ]:
var.to(unit='s')

Conversion also works the other way around:

In [ ]:
sc.units.aliases['dogyear'] = '4492800s'
sc.scalar(8985600, unit='s').to(unit='dogyear')